### 图注意力神经网络（GAT）

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GAT(nn.Module):

    def __init__(self, num_features, num_classes, hidden_channels):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) # [num_features, num_classes]
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

### 图注意神经网络的训练

In [2]:
from torch_geometric.datasets import Planetoid
# 加载数据集
dataset = Planetoid(root='dataset', name='Cora')
# 获取第一张图
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

model = GAT(num_features=dataset.num_features, num_classes=dataset.num_features, hidden_channels=16)
print(model)

# 定义优化函数
optimizer = Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# 定义损失函数
criterion = CrossEntropyLoss()


def train():
    model.train()
    # 初始化权重
    optimizer.zero_grad()
    # forward
    out = model(data.x, data.edge_index)
    # 计算loss
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    # 反向传播
    loss.backward()
    # 更新梯度
    optimizer.step()
    return loss

GAT(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 1433)
)


In [7]:
from tqdm import tnrange
import sys

for epoch in tnrange(10):
    best_loss = sys.maxsize
    for iteration in range(100):
        loss = train()
        if loss < best_loss:
            best_loss = loss
    print(f"Epoch : {epoch} , Loss : {best_loss:.4f}")

D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch : 0 , Loss : 0.0627
Epoch : 1 , Loss : 0.0429
Epoch : 2 , Loss : 0.0297
Epoch : 3 , Loss : 0.0276
Epoch : 4 , Loss : 0.0189
Epoch : 5 , Loss : 0.0200
Epoch : 6 , Loss : 0.0096
Epoch : 7 , Loss : 0.0141
Epoch : 8 , Loss : 0.0122
Epoch : 9 , Loss : 0.0107


### 图注意力神经网络的测试

In [9]:
def test():
    model.eval()
    # 获取训练后的模型输出
    out = model(data.x, data.edge_index)
    # 获取预测结果
    pred = out.argmax(dim=1)
    test_correct = pred[data.test_mask] == data.y[data.test_mask]
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    return test_acc

In [10]:
test_acc = test()
print(f"准确度 : {test_acc:.4f}")

准确度 : 0.8060
